<a href="https://colab.research.google.com/github/Anjasfedo/eceg-lsb-lzw-huffman/blob/main/image_compression_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import os
import numpy as np

class LZW:
    def __init__(self, path):
        self.path = path
        self.compressionDictionary, self.compressionIndex = self.createCompressionDict()
        self.decompressionDictionary, self.decompressionIndex = self.createDecompressionDict()

    ''''''
    ''' --------------------- Compression of the Image --------------------- '''
    ''''''

    def compress(self):
        self.initCompress()
        compressedcColors = []
        print("Compressing Image ...")
        compressedcColors.append(self.compressColor(self.red))
        print("Compressing Image ...")
        compressedcColors.append(self.compressColor(self.green))
        print("Compressing Image ...")
        compressedcColors.append(self.compressColor(self.blue))
        print("Image Compressed --------- Writing to File")
        filesplit = str(os.path.basename(self.path)).split('.')
        filename = filesplit[0] + 'Compressed.lzw'
        savingDirectory = os.path.join(os.getcwd(),'CompressedFiles')
        if not os.path.isdir(savingDirectory):
            os.makedirs(savingDirectory)
        with open(os.path.join(savingDirectory,filename),'w') as file:
            for color in compressedcColors:
                for row in color:
                    file.write(row)
                    file.write("\n")

    def compressColor(self, colorList):
        compressedColor = []
        i = 0
        for currentRow in colorList:
            currentString = currentRow[0]
            compressedRow = ""
            i+=1
            for charIndex in range(1, len(currentRow)):
                currentChar = currentRow[charIndex]
                if currentString+currentChar in self.compressionDictionary:
                    currentString = currentString+currentChar
                else:
                    compressedRow = compressedRow + str(self.compressionDictionary[currentString]) + ","
                    self.compressionDictionary[currentString+currentChar] = self.compressionIndex
                    self.compressionIndex += 1
                    currentString = currentChar
                currentChar = ""
            compressedRow = compressedRow + str(self.compressionDictionary[currentString])
            compressedColor.append(compressedRow)
        return compressedColor

    ''''''
    ''' --------------------- Deompression of the Image --------------------- '''
    ''''''

    def decompress(self):
        print("Decompressing File ...")
        image = []
        with open(self.path,"r") as file:
            for line in file:
                decodedRow = self.decompressRow(line)
                image.append(np.array(decodedRow))
        image = np.array(image)
        shapeTup = image.shape
        image = image.reshape((3,shapeTup[0]//3,shapeTup[1]))
        self.saveImage(image)
        print("Decompression Done.")

    def decompressRow(self,line):
        currentRow = line.split(",")
        currentRow[-1] = currentRow[-1][:-1]
        decodedRow = ""
        word,entry = "",""
        decodedRow = decodedRow + self.decompressionDictionary[int(currentRow[0])]
        word = self.decompressionDictionary[int(currentRow[0])]
        for i in range(1,len(currentRow)):
            new = int(currentRow[i])
            if new in self.decompressionDictionary:
                entry = self.decompressionDictionary[new]
                decodedRow += entry
                add = word + entry[0]
                word = entry
            else:
                entry = word + word[0]
                decodedRow += entry
                add = entry
                word = entry
            self.decompressionDictionary[self.decompressionIndex] = add
            self.decompressionIndex+=1
        newRow = decodedRow.split(',')
        decodedRow = [int(x) for x in newRow]
        return decodedRow

    ''''''
    ''' ---------------------- Class Helper Functions ---------------------- '''
    ''''''

    '''
    Used For: Compression of Image
    Function: This function breaks down the image into the three constituting
              image chanels - Red, Green and Blue.
    '''
    def initCompress(self):
        self.image = Image.open(self.path)
        self.height, self.width = self.image.size
        self.red, self.green, self.blue = self.processImage()

    '''
    Used For: Compression of Image
    Function: This function breaks down the image into the three constituting
              image chanels - Red, Green and Blue.
    '''
    def processImage(self):
        image = self.image.convert('RGB')
        red, green, blue = [], [], []
        pixel_values = list(image.getdata())
        iterator = 0
        for height_index in range(self.height):
            R, G, B = "","",""
            for width_index in range(self.width):
                RGB = pixel_values[iterator]
                R = R + str(RGB[0]) + ","
                G = G + str(RGB[1]) + ","
                B = B + str(RGB[2]) + ","
                iterator+=1
            red.append(R[:-1])
            green.append(G[:-1])
            blue.append(B[:-1])
        return red,green,blue


    '''
    Used For: Decompression of Image
    Function: This function will save the decompressed image as <name>.png
    '''
    def saveImage(self,image):
        print("Saving Decompressed File...")
        filesplit = str(os.path.basename(self.path)).split('Compressed.lzw')
        filename = filesplit[0] + "Decompressed.png"
        savingDirectory = os.path.join(os.getcwd(),'DecompressedFiles')
        if not os.path.isdir(savingDirectory):
            os.makedirs(savingDirectory)
        imagelist,imagesize = self.makeImageData(image[0],image[1],image[2])
        imagenew = Image.new('RGB',imagesize)
        imagenew.putdata(imagelist)
        imagenew.save(os.path.join(savingDirectory,filename))

    '''
    Used For: Decompression of Image
    Function: This function will convert and return the image in the (r,g,b) format
              to save the image.
    '''
    def makeImageData(self,r,g,b):
        imagelist = []
        for i in range(len(r)):
            for j in range(len(r[0])):
                imagelist.append((r[i][j],g[i][j],b[i][j]))
        return imagelist,(len(r),len(r[0]))

    '''
    Used For: Compression of Image
    Function: This function will initialise the compression dictionary
    '''
    def createCompressionDict(self):
        dictionary = {}
        for i in range(10):
            dictionary[str(i)] = i
        dictionary[','] = 10
        return dictionary,11

    '''
    Used For: Compression of Image
    Function: This function will initialise the decompression dictionary
    '''
    def createDecompressionDict(self):
        dictionary = {}
        for i in range(10):
            dictionary[i] = str(i)
        dictionary[10] = ','
        return dictionary,11

In [ ]:
import heapq
import os

# Helper functions for reading and writing images
def read_image_bit_string(path):
    with open(path, 'rb') as image:
        bit_string = ""
        byte = image.read(1)
        while (len(byte) > 0):
            byte = ord(byte)
            bits = bin(byte)[2:].rjust(8, '0')
            bit_string += bits
            byte = image.read(1)
    return bit_string


def write_image(bit_string, path):
    with open(path, 'wb') as image:
        for i in range(0, len(bit_string), 8):
            byte = bit_string[i:i + 8]
            image.write(bytes([int(byte, 2)]))


def write_dictionary_file(dictionary, path):
    with open(path, 'w') as f:
        for key, value in dictionary.items():
            f.write('%s:%s\n' % (key, value))


# Huffman coding related functions
class node:
    def __init__(self, frequency, symbol, left=None, right=None):
        self.frequency = frequency
        self.symbol = symbol
        self.left = left
        self.right = right
        self.huffman_direction = ''

    def __lt__(self, nxt):
        return self.frequency < nxt.frequency


huffman_codes = {}

def get_compressed_image(image_bit_string):
    compressed_image_bit_string = ""
    for i in range(0, len(image_bit_string), 8):
        byte = image_bit_string[i:i + 8]
        compressed_image_bit_string += huffman_codes[byte]
    return compressed_image_bit_string


def calculate_huffman_codes(node, code=''):
    code += node.huffman_direction
    if node.left:
        calculate_huffman_codes(node.left, code)
    if node.right:
        calculate_huffman_codes(node.right, code)
    if not node.left and not node.right:
        huffman_codes[node.symbol] = code
    return huffman_codes


def get_merged_huffman_tree(byte_to_frequency):
    huffman_tree = []
    for byte, frequency in byte_to_frequency.items():
        heapq.heappush(huffman_tree, node(frequency, byte))
    while len(huffman_tree) > 1:
        left = heapq.heappop(huffman_tree)
        right = heapq.heappop(huffman_tree)
        left.huffman_direction = "0"
        right.huffman_direction = "1"
        merged_node = node(left.frequency + right.frequency,
                           left.symbol + right.symbol, left, right)
        heapq.heappush(huffman_tree, merged_node)
    return huffman_tree[0]


def get_frequency(image_bit_string):
    byte_to_frequency = {}
    for i in range(0, len(image_bit_string), 8):
        byte = image_bit_string[i:i + 8]
        if byte not in byte_to_frequency:
            byte_to_frequency[byte] = 0
        byte_to_frequency[byte] += 1
    return byte_to_frequency


def compress(image_bit_string):
    byte_to_frequency = get_frequency(image_bit_string)
    merged_huffman_tree = get_merged_huffman_tree(byte_to_frequency)
    calculate_huffman_codes(merged_huffman_tree)
    return get_compressed_image(image_bit_string)


def decompress(compressed_image_bit_string):
    decompressed_image_bit_string = ""
    current_code = ""
    for bit in compressed_image_bit_string:
        current_code += bit
        for byte, code in huffman_codes.items():
            if current_code == code:
                decompressed_image_bit_string += byte
                current_code = ""
    return decompressed_image_bit_string


# Main function to handle compression and decompression
def process_image(image_path, compressed_image_path, decompressed_image_path):
    # Ensure the output directories exist
    compressed_output_dir = os.path.dirname(compressed_image_path)
    decompressed_output_dir = os.path.dirname(decompressed_image_path)

    os.makedirs(compressed_output_dir, exist_ok=True)
    os.makedirs(decompressed_output_dir, exist_ok=True)

    # Read image and get its bit string
    image_bit_string = read_image_bit_string(image_path)

    # Compress the image
    compressed_image_bit_string = compress(image_bit_string)
    write_image(compressed_image_bit_string, compressed_image_path)

    # Print the compression ratio
    print("Compression Ratio (CR):", len(image_bit_string) / len(compressed_image_bit_string))

    # Decompress the image
    decompressed_image_bit_string = decompress(compressed_image_bit_string)
    write_image(decompressed_image_bit_string, decompressed_image_path)

    return compressed_image_path, decompressed_image_path


In [ ]:
name = 'lena'

In [ ]:
import os

compressor = LZW(f'{name}.png')
compressor.compress()

decompressor = LZW(os.path.join("CompressedFiles",f"{name}Compressed.lzw"))
decompressor.decompress()

Compressing Image ...
Compressing Image ...
Compressing Image ...
Image Compressed --------- Writing to File
Decompressing File ...
Saving Decompressed File...
Decompression Done.


In [ ]:
# Paths
original_image_path = f"{name}.png"
compressed_image_path = f"./compressed_image_{name}.bin"
decompressed_image_path = f"./decompressed_image_{name}.png"

# Call the process_image function with the specified paths
compress_path, decompress_path = process_image(original_image_path, compressed_image_path, decompressed_image_path)

# Print the results
print(f"Compressed image saved at: {compress_path}")
print(f"Decompressed image saved at: {decompress_path}")

Compression Ratio (CR): 1.0
Compressed image saved at: ./compressed_image_lena.bin
Decompressed image saved at: ./decompressed_image_lena.png


In [ ]:
original_image_path

'lena.png'

In [ ]:
import os
import heapq
from google.colab import files

# Helper functions for file handling
def read_image_bit_string(image_path):
    bit_string = ""
    with open(image_path, 'rb') as image:
        byte = image.read(1)
        while byte:
            byte_value = ord(byte)
            bits = bin(byte_value)[2:].rjust(8, '0')
            bit_string += bits
            byte = image.read(1)
    return bit_string


def save_data(bit_string_or_dictionary, save_path, data_type):
    # Save data to the specified path
    if data_type == 'image':
        with open(save_path, 'wb') as file:
            for i in range(0, len(bit_string_or_dictionary), 8):
                byte_bits = bit_string_or_dictionary[i:i + 8]
                file.write(bytes([int(byte_bits, 2)]))
    elif data_type == 'dictionary':
        with open(save_path, 'w') as file:
            for key, value in bit_string_or_dictionary.items():
                file.write('%s:%s\n' % (key, value))
    else:
        raise ValueError("Invalid data_type. Supported values: 'image', 'dictionary'")


# Huffman coding implementation
class Node:

    def __init__(self, frequency, symbol, left=None, right=None):
        self.frequency = frequency
        self.symbol = symbol
        self.left = left
        self.right = right
        self.huffman_direction = ''

    def __lt__(self, nxt):
        return self.frequency < nxt.frequency


huffman_codes = {}

def calculate_huffman_codes(node, code=''):
    code += node.huffman_direction
    if node.left:
        calculate_huffman_codes(node.left, code)
    if node.right:
        calculate_huffman_codes(node.right, code)
    if not node.left and not node.right:
        huffman_codes[node.symbol] = code


def get_merged_huffman_tree(byte_to_frequency):
    huffman_tree = [Node(freq, byte) for byte, freq in byte_to_frequency.items()]
    heapq.heapify(huffman_tree)
    while len(huffman_tree) > 1:
        left = heapq.heappop(huffman_tree)
        right = heapq.heappop(huffman_tree)
        left.huffman_direction = "0"
        right.huffman_direction = "1"
        merged_node = Node(left.frequency + right.frequency, left.symbol + right.symbol, left, right)
        heapq.heappush(huffman_tree, merged_node)
    return huffman_tree[0]


def get_frequency(image_bit_string):
    byte_to_frequency = {}
    for i in range(0, len(image_bit_string), 8):
        byte = image_bit_string[i:i + 8]
        byte_to_frequency[byte] = byte_to_frequency.get(byte, 0) + 1
    return byte_to_frequency


def compress(image_bit_string):
    byte_to_frequency = get_frequency(image_bit_string)
    merged_huffman_tree = get_merged_huffman_tree(byte_to_frequency)
    calculate_huffman_codes(merged_huffman_tree)

    compressed_image_bit_string = ""
    for i in range(0, len(image_bit_string), 8):
        byte = image_bit_string[i:i + 8]
        compressed_image_bit_string += huffman_codes[byte]

    return compressed_image_bit_string


def decompress(compressed_image_bit_string):
    decompressed_image_bit_string = ""
    current_code = ""
    code_to_byte = {code: byte for byte, code in huffman_codes.items()}

    for bit in compressed_image_bit_string:
        current_code += bit
        if current_code in code_to_byte:
            decompressed_image_bit_string += code_to_byte[current_code]
            current_code = ""
    return decompressed_image_bit_string


# Main function to compress and decompress
def process_image():
    # Upload the original image
    print("Please upload the image to compress:")
    uploaded = files.upload()

    for image_name in uploaded.keys():
        image_path = image_name

    # Read the image bit string
    image_bit_string = read_image_bit_string(image_path)

    # Compress the image
    compressed_image_bit_string = compress(image_bit_string)
    compressed_image_path = "compressed_image.bin"
    save_data(compressed_image_bit_string, compressed_image_path, 'image')
    print(f"Compressed image saved as {compressed_image_path}")
    files.download(compressed_image_path)

    # Decompress the image
    decompressed_image_bit_string = decompress(compressed_image_bit_string)
    decompressed_image_path = "decompressed_image.png"
    save_data(decompressed_image_bit_string, decompressed_image_path, 'image')
    print(f"Decompressed image saved as {decompressed_image_path}")
    files.download(decompressed_image_path)


# Run the process
process_image()


Please upload the image to compress:


Saving lena.png to lena (1).png
Compressed image saved as compressed_image.bin


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Decompressed image saved as decompressed_image.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from PIL import Image
import heapq
import time

'''
Things this program does:
    1. Loads an image and pixel data as an object of the Picture class
    2. Creates a dictionary of [R,G,B: frequency] for each pixel
    3. Creates a heap of dictionary elements
    4. Recursively merges heap elements into a huff tree
    5. Traverses huff tree to generated encoded strings
    6. Refills dictionary with [R,G,B: encoded strings]
    7. Produces text file containing compressed image data

'''


class HeapNode:
    def __init__(self, rgb, freq):
        self.rgb = rgb
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

    def __eq__(self, other):
        if other == None:
            return False
        if(not isinstance(other, HeapNode)):
            return False
        return self.freq == other.freq


class Picture:
    def __init__(self, name):
        self.name = name[:-4]
        self.dict = {}
        self.data = []
        self.heap = []
        self.encode = {}
        self.decode = {}
        self.res_image = []
        self.recieve = []

        self.img = Image.open(name)
        self.width, self.height = self.img.size
        self.px = self.img.load()

    def load_data(self):
        for row in range(self.width):
            for col in range(self.height):
                self.data.append(self.px[row, col])
        for item in self.data:
            if item not in self.dict:
                self.dict[item] = 1
            else:
                self.dict[item] += 1

    def display(self, img_object):
        img_object.show()

    def get_data(self):
        return self.data

    def get_dict(self):
        return self.dict

    def make_heap(self):
        for key in self.dict:
            if self.dict[key] > 0:
                node = HeapNode(key, self.dict[key])
                heapq.heappush(self.heap, node)

    def merge_nodes(self):
        while(len(self.heap)>1):
            node_one = heapq.heappop(self.heap)
            node_two = heapq.heappop(self.heap)

            merge = HeapNode(None, node_one.freq + node_two.freq)
            merge.left = node_one
            merge.right = node_two

            heapq.heappush(self.heap, merge)

    def heaporder(self, root, buffer):
        if root:
            self.res_image.append([root.rgb, root.freq, buffer])
            buffer += "0"
            self.heaporder(root.left, buffer)
            buffer = buffer[:-1]
            buffer += "1"
            self.heaporder(root.right, buffer)

    def create_compression_keys(self):
        for item in self.res_image:
            if item[0]:
                self.encode[item[0]] = item[2]
                self.decode[item[2]] = item[0]

    def writeout(self):
        with open(self.name+"_out.png", 'w') as out:
            for pix in self.data:
                out.write(self.encode[pix]+"\n")

    def readin(self):
        with open(self.name+"_out.png", 'r') as ins:
            self.recieve = ins.read().splitlines()

    def create_new_image(self):
        decompressed = Image.new('RGB', (self.width, self.height))
        pixels = decompressed.load()
        index = 0
        for row in range(self.width):
            for col in range(self.height):
                pixels[row, col] = self.decode[self.recieve[index]]
                index += 1
        self.display(decompressed)


numba_one = Picture(input("Filename:"))
print("Thinking . . .")
start_c = time.time()
numba_one.load_data()
print("Image size:", len(numba_one.data))
numba_one.make_heap()
numba_one.merge_nodes()
numba_one.heaporder(numba_one.heap[0], "")
numba_one.create_compression_keys()
end_c = time.time()
duration_c = (end_c-start_c)*1000
print("Compression:", duration_c, "milliseconds.")
numba_one.writeout()
numba_one.readin()

start_d = time.time()
numba_one.create_new_image()
end_d = time.time()
duration_d = (end_d - start_d)*1000
print("Decompression:", duration_d, "milliseconds.")


Filename:lena.png
Thinking . . .
Image size: 262144
Compression: 3864.2709255218506 milliseconds.
Decompression: 290.29297828674316 milliseconds.


In [ ]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer

# Load the model
model2 = TFBertForSequenceClassification.from_pretrained("/content/model_output")

# Load the tokenizer
tokenizer2 = BertTokenizer.from_pretrained("/content/model_output")

# Prediction function using TensorFlow
def predict(text):
    inputs = tokenizer2(text, return_tensors='tf', truncation=True, padding=True, max_length=512)

    # Get logits from model2
    outputs = model2(inputs)
    logits = outputs.logits

    # Apply softmax to get probabilities
    probs = tf.nn.softmax(logits, axis=-1)

    return tf.argmax(probs, axis=-1).numpy()[0]  # Get the predicted label index

# Test sample text
forum_tittle = "Strategi Debugging dan Penanganan Error dalam Pemrograman"
forum_description = "Diskusi ini bertujuan untuk membahas cara mengenali, memahami, dan menyelesaikan error dalam kode. Mari berbagi tips tentang debugging, penggunaan tools, dan best practices untuk mengatasi error secara efisien."
forum_message = "Menurut saya, struktur log yang tersusun dengan baik mempermudah dalam mengidentifikasi pola error di aplikasi berbasis cloud."
sample_text = forum_tittle + "|" + forum_description + " " + forum_message

# Predict and print label
label_map = {0: "C1", 1: "C2", 2: "C3", 3: "C4", 4: "C5", 5: "C6"}
predicted_label = predict(sample_text)
print(f"Label: {label_map[predicted_label]}")

In [ ]:
Strategi Debugging dan Penanganan Error dalam Pemrograman Diskusi ini bertujuan untuk membahas cara mengenali, memahami, dan menyelesaikan error dalam kode. Mari berbagi tips tentang debugging, penggunaan tools, dan best practices untuk mengatasi error secara efisien. Menurut saya, struktur log yang tersusun dengan baik mempermudah dalam mengidentifikasi pola error di aplikasi berbasis cloud.

In [ ]:
title, desc